#### -----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
import time
from arena_api.system import system

#### Acquisition: Introduction
>    This example introduces the basics of image acquisition. This includes
    setting image acquisition and buffer handling modes, setting the device to
    automatically negotiate packet size, and setting the stream packet resend
    node before starting the image stream. The example then starts acquiring
    images by grabbing and requeuing buffers, and retrieving data on images
    before finally stopping the image stream.

In [ ]:
"""
This function waits for the user to connect a device before raising an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
            tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        device = devices[0]
        nodemap = device.nodemap
        tl_stream_nodemap = device.tl_stream_nodemap
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')


##### Store initial values
> These initial values are restored to the device after the example is completed

In [ ]:
print(f"Store initial values")
initial_acquisition_mode = nodemap.get_node("AcquisitionMode").value
width_initial = nodemap.get_node("Width").value
height_initial = nodemap.get_node("Height").value

##### Configure device settings

In [ ]:
nodemap.get_node("AcquisitionMode").value = "Continuous"

##### Set buffer handling mode
>  Set buffer handling mode before starting the stream. Starting the
    stream requires the buffer handling mode to be set beforehand. The
    buffer handling mode determines the order and behavior of buffers in
    the underlying stream engine. Setting the buffer handling mode to
    'NewestOnly' ensures the most recent image is delivered, even if it
    means skipping frames.

In [ ]:
tl_stream_nodemap["StreamBufferHandlingMode"].value = "NewestOnly"

##### Enable stream auto negotiate packet size
> Setting the stream packet size is done before starting the stream.
    Setting the stream to automatically negotiate packet size instructs the
    camera to receive the largest packet size that the system will allow.
    This generally increases frame rate and results in fewer interrupts per
    image, thereby reducing CPU load on the host system. Ethernet settings
    may also be manually changed to allow for a larger packet size.

In [ ]:
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True

##### Enable stream packet resend
> Enable stream packet resend before starting the stream. Images are sent
    from the camera to the host in packets using UDP protocol, which
    includes a header image number, packet number, and timestamp
    information. If a packet is missed while receiving an image, a packet
    resend is requested and this information is used to retrieve and
    redeliver the missing packet in the correct order.

In [ ]:
tl_stream_nodemap['StreamPacketResendEnable'].value = True

##### Set features before streaming.-------------------------------------------
>  Set width and height to their max values

In [ ]:
print(f'Getting \'Width\' and \'Height\' Nodes')
nodes = device.nodemap.get_node(['Width', 'Height'])

print('Setting \'Width\' and \'Height\' Nodes value to their '
      'max values')
nodes['Width'].value = nodes['Width'].max
nodes['Height'].value = nodes['Height'].max


##### Grab images -------------------------------------------------------------
> - Starting the stream allocates buffers, which can be passed in as
 an argument (default: 10), and begins filling them with data.
 Buffers must later be requeued to avoid memory leaks.<br>
> - 'Device.get_buffer()' with no arguments returns one buffer(NOT IN A LIST)<br>
> - 'Device.get_buffer(30)' returns 30 buffers(IN A LIST)<br>
> - 'Device.requeue_buffer()' takes a buffer or many buffers in a list or tuple

In [ ]:
number_of_buffers = 10

device.start_stream(number_of_buffers)
print(f'Stream started with {number_of_buffers} buffers')

print(f'Get {number_of_buffers} buffers in a list')
buffers = device.get_buffer(number_of_buffers)
print("Success")

'''
Print image buffer info
    Buffers contain image data.
    Image data can also be copied and converted using BufferFactory.
    That is necessary to retain image data, as we must also requeue the buffer.
'''
for count, buffer in enumerate(buffers):
    print(f'\tbuffer{count:{2}} received | '
        f'Width = {buffer.width} pxl, '
        f'Height = {buffer.height} pxl, '
        f'Pixel Format = {buffer.pixel_format.name}')

device.requeue_buffer(buffers)
print(f'Requeued {number_of_buffers} buffers')

device.stop_stream()
print(f'Stream stopped')

##### Clean up ----------------------------------------------------------------
> - Restore initial values to the device.
> - Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [ ]:
nodemap.get_node("AcquisitionMode").value = initial_acquisition_mode
nodemap.get_node("Width").value = width_initial
nodemap.get_node("Height").value = height_initial

system.destroy_device()
print('Destroyed all created devices')